In [1]:
import numpy as np

from src.vrp_study.data_model import Tariff, Cargo, Node
from src.vrp_study.data_model import TariffCost
from src.vrp_study.pdptw_model.pdptw_routing_manager_builder import PDRoutingManagerBuilder
from ortools.sat.python.cp_model import CpSolver
from src.vrp_study.pdptw_model.routing_model import find_optimal_paths

In [2]:
PRINT_LOG = False

In [3]:
best = {'preferred_variable_order': 2,
        'clause_cleanup_protection': 1,
        'max_presolve_iterations': 5,
        'cp_model_probing_level': 1,
        'presolve_probing_deterministic_time_limit': 1.0,
        'search_branching': 2,
        'feasibility_jump_linearization_level': 0,
        'fp_rounding': 0,
        'polish_lp_solution': True,
        'linearization_level': 0,
        'cut_level': 2,
        'max_all_diff_cut_size': 128,
        'symmetry_level': 0,
        'num_workers': 4}


def get_solver():
    solver = CpSolver()

    for k, v in best.items():
        if isinstance(v, list):
            for ss in v:
                solver.parameters.ignore_subsolvers.append(ss)
        else:
            if 'ignore_subsolvers' in k:
                if v:
                    solver.parameters.ignore_subsolvers.append(k.split(':')[1])
            else:
                exec(f'solver.parameters.{k} = {v}')
    # solver.parameters.use_lns = True
    # solver.parameters.lns_num_threads = 4
    solver.parameters.log_search_progress = PRINT_LOG
    solver.parameters.max_time_in_seconds = 60.0 * 10

    # packing_subsolver = sat_parameters_pb2.SatParameters()
    # packing_subsolver.name = "MyPackingSubsolver"
    # packing_subsolver.use_area_energetic_reasoning_in_no_overlap_2d = False
    # packing_subsolver.use_energetic_reasoning_in_no_overlap_2d = False
    # packing_subsolver.use_timetabling_in_no_overlap_2d = False
    # packing_subsolver.max_pairs_pairwise_reasoning_in_no_overlap_2d = 5_000
    # packing_subsolver.
    # # Add the subsolver to the portfolio
    # solver.parameters.subsolver_params.append(packing_subsolver)  # Define the subsolver
    # solver.parameters.extra_subsolvers.append(
    #     packing_subsolver.name
    # )  # Activate the subsolver
    return solver

In [4]:
benchmark_type = 'pdp_400'
name = 'LC1_4_9.txt'

In [5]:
from typing import Optional

tariff = None
cargos: list[Cargo] = []
depo: Optional[Node] = None

In [6]:
id2info = {}
p2coordinates = {}
with open(f'../data/Li & Lim benchmark/{benchmark_type}/{name}', 'r') as file:
    for i, line in enumerate(file):
        line = line.split('\t')
        if i == 0:
            tariff = Tariff(
                id='car',
                capacity=int(line[1]),
                max_count=int(line[0]),
                cost_per_distance=[TariffCost(
                    min_dst_km=0,
                    max_dst_km=10000,
                    cost_per_km=1,
                    fixed_cost=0
                )]
            )
        else:
            c_id = int(line[0])
            x = int(line[1])
            y = int(line[2])

            mass = int(line[3])

            et = int(line[4])
            lt = int(line[5])
            st = int(line[6])

            pick_up = int(line[7])
            delivery = int(line[8])
            if pick_up == delivery:
                # print(12)
                depo = Node(
                    id=0,
                    cargo_id=c_id,
                    capacity=0,
                    service_time=0,
                    start_time=0,
                    end_time=lt,
                    coordinates=(x, y)
                )
                continue
            if pick_up == 0:
                if c_id not in id2info:
                    id2info[c_id] = {}
                id2info[c_id][0] = (x, y, mass, et, lt, st, c_id, delivery)
            else:
                delivery = c_id
                c_id = pick_up
                if c_id not in id2info:
                    id2info[c_id] = {}
                id2info[c_id][1] = (x, y, mass, et, lt, st, pick_up, delivery)


In [7]:
depo

Node(id=0, cargo_id=0, capacity=0, start_time=0, end_time=1501, service_time=0, coordinates=(100, 100))

In [8]:

for k, v in id2info.items():
    cargos.append(
        Cargo(
            id=k,
            nodes=[
                Node(
                    cargo_id=k,
                    id=v[i][6] if i == 0 else v[i][7],
                    capacity=v[i][2],
                    service_time=v[i][5],
                    start_time=v[i][3],
                    end_time=v[i][4],
                    coordinates=(v[i][0], v[i][1])
                )
                for i in range(2)
            ]
        )
    )

In [9]:
p2coordinates.update({
    crg.nodes[i].id: crg.nodes[i].coordinates for crg in cargos for i in range(2)
})
p2coordinates[depo.id] = depo.coordinates
distance_matrix = {(u, v): np.sqrt((du[0] - dv[0]) ** 2 + (du[1] - dv[1]) ** 2) for u, du in
                   p2coordinates.items() for
                   v, dv in p2coordinates.items()}
time_matrix = {(u, v): np.sqrt((du[0] - dv[0]) ** 2 + (du[1] - dv[1]) ** 2) for u, du in p2coordinates.items() for
               v, dv in p2coordinates.items()}

In [10]:
len(cargos)

209

In [22]:
cargos = cargos[:5]

In [23]:

from src.vrp_study.configs import ModelConfig

routing_manager = PDRoutingManagerBuilder(
    distance_matrix=distance_matrix,
    time_matrix=time_matrix,
    model_config=ModelConfig(max_execution_time_minutes=1)
)

routing_manager.add_cargos(cargos)
routing_manager.add_tariff(tariff)

routing_manager.add_depo(depo)

routing_manager = routing_manager.build()

In [24]:
from ortools.sat.python.cp_model import CpModel
from vrp_study.routing_manager import RoutingManager


class RoutingModel:
    def __init__(self, routing_manager: RoutingManager):
        self.N = 8
        self.M = 1000_000
        self.R = {}
        self.T = {}
        self.model = CpModel()
        self.routing_manager = routing_manager
        self.__init_model()

    def __init_model(self):
        N = self.N
        M = self.M
        R = self.R
        T = self.T
        model = self.model
        routing_manager = self.routing_manager

        for r in range(N):
            for i, n in enumerate(routing_manager.nodes()):
                R[r, i] = model.new_bool_var(f'r_{r, i}')

        min_time = min(n.start_time for n in routing_manager.nodes())
        max_time = max(n.end_time for n in routing_manager.nodes())

        for r in range(N):
            T[r] = model.new_int_var(lb=min_time, ub=max_time, name=f'time_{r}')

        nodes = routing_manager.nodes()

        for r in range(N):
            model.add(sum(R[r, i] for i in range(len(nodes))) == 1)

        for i, n in enumerate(nodes):
            if n.is_transit:
                model.add(sum(R[r, i] for r in range(N)) <= 1)

        for r in range(N):
            model.add(T[r] <= sum(R[r, i] * n.end_time for i, n in enumerate(nodes)))
            model.add(T[r] >= sum(R[r, i] * n.start_time for i, n in enumerate(nodes)))

        for r in range(N - 1):
            for j, nj in enumerate(nodes):
                model.add(T[r + 1] - T[r] >= sum(
                    R[r, i] * int(ni.service_time + routing_manager.get_time(ni, nj)) for i, ni in enumerate(nodes)
                ) - M * (1 - R[r + 1, j]))

        for pdp in routing_manager.get_pick_up_and_delivery_nodes():
            indices = sum([[R[r, i] for i in pdp] for r in range(N)], [])
            transitions = [
                (0, 0, 1),
                (1, 0, 0),
                (0, 1, 2),
                (2, 0, 2),
                (2, 1, 3),
                (3, 0, 3)
            ]
            model.add_automaton(
                transition_triples=transitions,
                transition_expressions=indices,
                starting_state=0,
                final_states=[3, 0],
            )
            # indices = sum([[(r, i) for r in range(N)] for i in pdp], [])
            # print(indices)
            # print(transitions)

        indices = sum([[R[r, 0] for r in range(1, N)]], [])
        print(indices)
        transitions = [
            (0, 0, 0),
            (0, 1, 1),
            (1, 1, 1)
        ]
        model.add_automaton(
            transition_triples=transitions,
            transition_expressions=indices,
            starting_state=0,
            final_states=[1]
        )

        model.add(R[0, 0] == 1)
        model.add(R[N - 1, 0] == 1)


In [25]:
model = RoutingModel(routing_manager)

[r_(1, 0)(0..1), r_(2, 0)(0..1), r_(3, 0)(0..1), r_(4, 0)(0..1), r_(5, 0)(0..1), r_(6, 0)(0..1), r_(7, 0)(0..1)]


In [26]:
car = routing_manager.cars()[0]
car.start_node

InnerNode(id=0, start_time=0, end_time=1501, service_time=0, demand=0, is_transit=False, pdp_id=-1, routing_node=Node(id=0, cargo_id=0, capacity=0, start_time=0, end_time=1501, service_time=0, coordinates=(100, 100)))

In [27]:
car.end_node

InnerNode(id=0, start_time=0, end_time=1501, service_time=0, demand=0, is_transit=False, pdp_id=-1, routing_node=Node(id=0, cargo_id=0, capacity=0, start_time=0, end_time=1501, service_time=0, coordinates=(100, 100)))

In [28]:
model.model.maximize(
    sum(model.R[r, i] for r in range(model.N) for i, n in enumerate(routing_manager.nodes()) if n.id > 0)
)

In [29]:
# solver = get_solver()
# solver.solve(model.model)

In [30]:
visited_nodes = set()

In [31]:
# for r in range(model.N):
#     arr = [solver.value(model.R[r,i]) for i in range(len(routing_manager.nodes()))]
#     visited_nodes.update({i for i,val in enumerate(arr) if i > 0 and val == 1 })
#     for i in range(len(arr)):
#         print(arr[i],end=' ')
#         if i == 0 or i%2==0:
#             if i == 0:
#                 print('->',end='')
#             print('| ',end='')            
#     print('\n')
#     # print([solver.value(model.R[r,i]) for i in range(len(routing_manager.nodes()))])

In [32]:
visited_nodes

set()

In [33]:
while (len(visited_nodes) != len(cargos) * 2):
    for r in range(model.N):
        for node in visited_nodes:
            model.model.add_assumption(model.R[r, node].Not())
    solver = get_solver()
    solver.solve(model.model)
    for r in range(model.N):
        arr = [solver.value(model.R[r, i]) for i in range(len(routing_manager.nodes()))]
        visited_nodes.update({i for i, val in enumerate(arr) if i > 0 and val == 1})
        for i in range(len(arr)):
            print(arr[i], end=' ')
            if i == 0 or i % 2 == 0:
                if i == 0:
                    print('->', end='')
                print('| ', end='')
        print('\n')
        # print([solver.value(model.R[r,i]) for i in range(len(routing_manager.nodes()))])
    print(len(visited_nodes))
    break

1 ->| 0 0 | 0 0 | 0 0 | 0 0 | 0 0 | 

0 ->| 0 0 | 0 0 | 0 0 | 0 0 | 1 0 | 

0 ->| 0 0 | 1 0 | 0 0 | 0 0 | 0 0 | 

0 ->| 0 0 | 0 0 | 0 0 | 0 0 | 0 1 | 

0 ->| 0 0 | 0 0 | 1 0 | 0 0 | 0 0 | 

0 ->| 0 0 | 0 0 | 0 1 | 0 0 | 0 0 | 

0 ->| 0 0 | 0 1 | 0 0 | 0 0 | 0 0 | 

1 ->| 0 0 | 0 0 | 0 0 | 0 0 | 0 0 | 

6


In [23]:
len(cargos) * 2

200

In [24]:
len(visited_nodes)

200

In [25]:
len(cargos)

100

In [26]:
# 9405